In [ ]:
# Configura conjuntos de treinamento e teste 
# X_train, X_test, Y_train, Y_test
%run './SLD-LoadTrainTestSets.ipynb'

In [ ]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Model, Input
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, GlobalAveragePooling2D, Reshape
from keras.optimizers import Adam

In [ ]:
original_input_shape = (64, 64, 3)
desired_input_shape = (64, 64, 1)
layer_name = 'conv1'

weights = None
include_top = False

---

# Adicionando camadas no final
_________________________________________________________________
global_average_pooling2d_6 ( (None, 1024)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 1, 1024)        0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 1024)        0         
_________________________________________________________________
conv_preds (Conv2D)          (None, 1, 1, 10)        ???   
_________________________________________________________________
act_softmax (Activation)     (None, 1, 1, 10)        0         

In [ ]:
# Configuração necessária para imagens 64x64
weights = None
include_top = False

In [ ]:
base_model = MobileNet(input_shape=original_input_shape,
                       weights=weights,
                       include_top=include_top)
base_model.layers[-1].name

In [ ]:
# Alterando input para grayscale (1 channel)
selected_layer = base_model.get_layer(layer_name)
selected_layer_index = base_model.layers.index(selected_layer)

w = selected_layer.get_weights()
w[0] = w[0].mean(axis=2, keepdims=True)

y = x = Input(shape=desired_input_shape)

attributes = dict(selected_layer.get_config(),
                  name=layer_name + '_1',
                  weights=w)
selected_layer = Conv2D.from_config(attributes)

y = selected_layer(y)

for layer in base_model.layers[selected_layer_index + 1:]:
    y = layer(y)

In [ ]:
# Adicionando camadas representadas pelo include_top=True
# camada FC no final da rede
y = GlobalAveragePooling2D()(y)
y = Reshape((1, 1, 1024))(y)
y = Dropout(0.25)(y)
y = Conv2D(10, kernel_size=(1, 1), activation='relu')(y)
y = Activation('softmax')(y)
y = Reshape((10,))(y)

In [ ]:
model = Model(inputs=x, outputs=y)
model.summary()

In [ ]:
len(model.layers)

In [ ]:
# Congela camadas
# for layer in model.layers:
#     layer.trainable = False

In [ ]:
adam_lr = 0.0001
nb_epochs = 4

cm_plot_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [ ]:
model.compile(Adam(lr=adam_lr), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

hist = model.fit(X_train, 
                 Y_train, 
                 validation_split=0.1, 
                 batch_size=10, 
                 epochs=nb_epochs, 
                 shuffle=True, 
                 verbose=1)

In [ ]:
model.compile(Adam(lr=adam_lr), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

new_X = expand_dims(X_train, axis=-1)

hist = model.fit(new_X, 
                 Y_train,
                 epochs=nb_epochs, 
                 steps_per_epoch=10,
                 shuffle=True, 
                 verbose=1)

---